In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import rand_score

In [2]:
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
time = pd.read_csv('time.csv')

In [3]:
X['time'] = time

In [4]:
# As you can see, apart from the time, these are just binary data.
X

,amateur+doggystyle,suck+ass,vegas,2+girls+share+cock,leia+lovelyn,ngentot,solo+joi,vagina+mojada,truutv,bjob,...,redhead+rough,rubbing+panties,2+girls+cum+mouth,italiane,forest,lights+off,ocean,antury,squirt+in+face,time
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,582
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,619
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,610
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,8063
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19190,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,510
19191,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,761
19192,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1071
19193,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1079


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
y_train.value_counts(normalize=True)

kolumna_celi
False           0.848333
True            0.151667
Name: proportion, dtype: float64

In [7]:
#It checks the independence of values, which is important in the case of binary data because, for example, there is a category "milf" and tags like "milf," "MILF," ..., "Milf".

chi2_stats, p_values = chi2(X_train, y_train)
chi2_stats

array([4.79756933e-04, 1.78782529e-01, 1.78782529e-01, ...,
       1.78782529e-01, 1.78782529e-01, 1.05767742e+03])

In [8]:
good_colums = []
for x, y in zip(chi2_stats,X_train.columns.tolist()):
  if x > 0.2:
    good_colums.append(y)
    
    
X_train_X = X_train[good_colums]
X_test_X = X_test[good_colums]

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [10]:
# The PCA algorithm doesn't work on binary columns, and MCA only reduces the interpretability of the data.

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  
    ('scaler', StandardScaler()),               
    ])

In [11]:
X_train_pipe = pipeline.fit_transform(X_train_X)
X_test_pipe = pipeline.fit_transform(X_test_X)

In [12]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [13]:
#There's no random forest because it simply doesn't learn, and no SVC because it takes too long.

classifiers = {
    'CatBoost': CatBoostClassifier(),
    'XGBoost': XGBClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
}

param_grids = {
    'DecisionTree': {
        'max_depth': [100, 200, 300, 400],
        'min_samples_split': [5]
    },
    'XGBoost': {
        'n_estimators': [150, 200, 300],
        'max_depth': [3, 6 ,9, 12],
        'learning_rate': [0.01, 0.1, 1, 2, 2.5],
        'subsample': [0.3, 0.6, 0.9]
    },
    'CatBoost': {
        'iterations': [100, 200, 300],
        'depth': [2, 4, 6, 8],
        'learning_rate': [0.01, 0.1, 1, 2, 2.5],
    }
}

In [14]:
# Why recall? Because since this is a detection problem, 
# it's very difficult to identify issues using only accuracy. 14% exceeded a million, 
# which means the model could give all zeros and still achieve 86% accuracy.

# best_estimators = {}
# for clf_name in classifiers:
#     print(f"Grid Search dla {clf_name}")
#     grid_search = GridSearchCV(classifiers[clf_name], param_grids[clf_name], cv=5, scoring='recall', n_jobs=-1)
#     grid_search.fit(X_train_pipe, y_train)
#     best_estimators[clf_name] = grid_search.best_estimator_
#     print(f"Najlepsze parametry dla {clf_name}: {grid_search.best_params_}")
#     print(f"Najlepszy wynik dla {clf_name}: {grid_search.best_score_}\n")

In [15]:
DT = DecisionTreeClassifier(max_depth= 200, min_samples_split = 5)
XGBC = XGBClassifier(learning_rate = 2, max_depth = 9, n_estimators = 150, subsample=0.6)
CBC = CatBoostClassifier(depth=8, iterations=300, learning_rate=2)

In [16]:
DT.fit(X_train_pipe, y_train)
XGBC.fit(X_train_pipe, y_train)
CBC.fit(X_train_pipe, y_train)

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


0:	learn: 0.5530114	total: 182ms	remaining: 54.4s
1:	learn: 0.6910769	total: 217ms	remaining: 32.3s
2:	learn: 0.5517031	total: 250ms	remaining: 24.8s
3:	learn: 0.6882917	total: 285ms	remaining: 21.1s
4:	learn: 0.5508004	total: 319ms	remaining: 18.8s
5:	learn: 0.6864182	total: 352ms	remaining: 17.2s
6:	learn: 0.5475267	total: 386ms	remaining: 16.1s
7:	learn: 0.6753214	total: 420ms	remaining: 15.3s
8:	learn: 0.5348932	total: 455ms	remaining: 14.7s
9:	learn: 0.6454660	total: 488ms	remaining: 14.2s
10:	learn: 0.5223465	total: 523ms	remaining: 13.7s
11:	learn: 0.6160093	total: 555ms	remaining: 13.3s
12:	learn: 0.5144943	total: 590ms	remaining: 13s
13:	learn: 0.6018484	total: 624ms	remaining: 12.7s
14:	learn: 0.4971520	total: 658ms	remaining: 12.5s
15:	learn: 0.5654418	total: 691ms	remaining: 12.3s
16:	learn: 0.4849976	total: 725ms	remaining: 12.1s
17:	learn: 0.5368061	total: 758ms	remaining: 11.9s
18:	learn: 0.4673213	total: 793ms	remaining: 11.7s
19:	learn: 0.5055892	total: 826ms	remaining

In [17]:
y_pred_DT = DT.predict(X_test_pipe)

print(recall_score(y_test, y_pred_DT))
print(precision_score(y_test, y_pred_DT))
print(confusion_matrix(y_test, y_pred_DT))

0.07653061224489796
0.16129032258064516
[[3017  234]
 [ 543   45]]


In [18]:
y_pred_XGBC = XGBC.predict(X_test_pipe)

print(recall_score(y_test, y_pred_XGBC))
print(precision_score(y_test, y_pred_XGBC))
print(confusion_matrix(y_test, y_pred_XGBC))

0.10034013605442177
0.17933130699088146
[[2981  270]
 [ 529   59]]


In [19]:
y_pred_CBC = CBC.predict(X_test_pipe)

print(recall_score(y_test, y_pred_CBC))
print(precision_score(y_test, y_pred_CBC))
print(confusion_matrix(y_test, y_pred_CBC))

0.12755102040816327
0.16375545851528384
[[2868  383]
 [ 513   75]]


In [ ]:
# I would choose XGBC because those two percentage points of recall are worth more than precision.

In [2]:
# Definitely create an NLP model based on titles
# Download thumbnails to check if it makes business sense based on them
# In the future, analyze watchtime, download participants/creators to see if it makes business sense
# Then integrate everything into a group model